### pySIPFENN MGF-PSU Workshop (Feb 2023)

This Jupyter notebook is a brief walkthrough covering core functionalities of the **pySIPFENN** or **py**(**S**tructure-**I**nformed **P**rediction of **F**ormation **E**nergy using **N**eural **N**etworks) package; available through the PyPI repository. A fuller description of capabilities is given at PSU Phases Research Lab webpage under _phaseslab.com/sipfenn_.

### Install pySIPFENN

Installing pySIPFENN is simple and easy utilizing either **PyPI** package repository or cloning from **GitHub**. While not required, it is recommended to first set up a virtual environement using venv or Conda. This ensures that one of the required versions of Python (3.9+) is used and there are no dependency conflicts. To create one

    conda create -n pysipfenn-workshop python=3.9 jupyter
    conda activate pysipfenn-workshop

And then install pySIPFENN from PyPI with

    pip install pysipfenn

Alternatively, you can also install pySIPFENN in editable mode if you cloned it from GitHub like

    git clone https://github.com/PhasesResearchLab/pySIPFENN.git

Or by downloading a ZIP file. Then, move to the pySIPFENN folder and install in editable (-e) mode

    cd pySIPFENN
    pip install -e .


### Starting pySIPFENN

To utilize pySIPFENN for straightforward calculations, only the Calculator class is needed. It allows for both fetching and identification of NN models and later running of them.

In [1]:
from pysipfenn import Calculator

Now initialize the Calculator. When run, this should display all models detected
(e.g. ✔ SIPFENN_Krajewski2020 Standard Materials Model)
and those not detected, but declared in the _modelsSIPFENN/models.json_ file.

In [2]:
c = Calculator()

✔ SIPFENN_Krajewski2020 Standard Materials Model
✔ SIPFENN_Krajewski2020 Novel Materials Model
✔ SIPFENN_Krajewski2020 Light Model
✔ SIPFENN_Krajewski2022 KS2022 Novel Materials Model
Loading models:


100%|██████████| 4/4 [00:13<00:00,  3.30s/it]

*********  PySIPFENN Successfully Initialized  **********


If this is the first run of pySIPFENN and no models are available, one can fetch four default (as of Feb 2023) models from Zenodo with a simple:

In [3]:
c.downloadModels()
c.loadModels()

Fetching all networks!
SIPFENN_Krajewski2020_NN9 detected on disk. Ready to use.
SIPFENN_Krajewski2020_NN20 detected on disk. Ready to use.
SIPFENN_Krajewski2020_NN24 detected on disk. Ready to use.
SIPFENN_Krajewski2022_NN30 detected on disk. Ready to use.
All networks available!
✔ SIPFENN_Krajewski2020 Standard Materials Model
✔ SIPFENN_Krajewski2020 Novel Materials Model
✔ SIPFENN_Krajewski2020 Light Model
✔ SIPFENN_Krajewski2022 KS2022 Novel Materials Model
Loading models:


100%|██████████| 4/4 [00:10<00:00,  2.59s/it]


For the purpose of testing, a single model is sufficient and will be fetched faster. E.g. the light model can be acquired in about 1/10 of the time.

In [ ]:
#c.downloadModels(network='SIPFENN_Krajewski2020_NN24')
#c.loadModels()

### Simple run

In [ ]:
#c.runFromDirectory(directory='myInputFiles', descriptor='KS2022')

In [6]:
c.runFromDirectory(directory='../pysipfenn/tests/testCaseFiles/exampleInputFiles/', descriptor='KS2022')

Importing structures...


100%|██████████| 32/32 [00:00<00:00, 128.20it/s]



Models that will be run: ['SIPFENN_Krajewski2022_NN30']
Calculating descriptors...


100%|██████████| 32/32 [00:07<00:00,  4.06it/s]

Done!
Making predictions...
Obtained predictions from:  SIPFENN_Krajewski2022_NN30
Done!


In [7]:
c.writeResultsToCSV('myResults.csv')

In [9]:
c.get_resultDictsWithNames()

[{'name': '0-Cr8Fe18Ni4.POSCAR',
  'SIPFENN_Krajewski2022_NN30': 0.17857088148593903},
 {'name': '1-Cr16Fe8Ni6.POSCAR',
  'SIPFENN_Krajewski2022_NN30': 0.22403044998645782},
 {'name': '2-Fe8Ni22.POSCAR',
  'SIPFENN_Krajewski2022_NN30': 0.07981749624013901},
 {'name': '3-Cr18Fe12.POSCAR',
  'SIPFENN_Krajewski2022_NN30': 0.30798229575157166},
 {'name': '4-Fe30.POSCAR', 'SIPFENN_Krajewski2022_NN30': 0.17809279263019562},
 {'name': '5-Cr22Fe8.POSCAR',
  'SIPFENN_Krajewski2022_NN30': 0.30708691477775574},
 {'name': '6-Fe2Ni28.POSCAR',
  'SIPFENN_Krajewski2022_NN30': 0.1531030684709549},
 {'name': '7-Cr18Fe12.POSCAR',
  'SIPFENN_Krajewski2022_NN30': 0.29845377802848816},
 {'name': '8-Cr2Fe16Ni12.POSCAR',
  'SIPFENN_Krajewski2022_NN30': 0.14972664415836334},
 {'name': '9-Pb8O12.POSCAR', 'SIPFENN_Krajewski2022_NN30': -1.293998122215271},
 {'name': '10-Ce4Ti4O12.POSCAR',
  'SIPFENN_Krajewski2022_NN30': -3.1219778060913086},
 {'name': '11-Fe10Ni20.POSCAR',
  'SIPFENN_Krajewski2022_NN30': 0.14550